In [38]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import re

In [39]:
url = f"https://www.transfermarkt.us/inter-miami-cf/startseite/verein/69261/saison_id/2024"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [40]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

address_blocks = soup.find_all("div", itemprop="address")

In [41]:
for block in address_blocks:
    address = {}

    # Extract structured address parts
    street = block.find(itemprop="streetAddress")
    locality = block.find(itemprop="addressLocality")
    postal_code = block.find(itemprop="postalCode")

    if street: address["street"] = street.get_text(strip=True)
    if locality: address["country"] = locality.get_text(strip=True)
    if postal_code: address["zip"] = postal_code.get_text(strip=True)
    print(address)

{'street': '1350 NW 55th St'}
{'zip': '33309 Fort Lauderdale, Florida'}
{'country': 'United States'}


In [42]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# STEP 1: Get team URLs from league page
def get_team_urls(league_url):
    res = requests.get(league_url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")


    team_links = soup.select('td.hauptlink.no-border-links a')
    
    team_urls = []
    for link in team_links:
        href = link.get("href")
        if href and "/startseite/verein/" in href:
            # Ensure full URL and keep ?saison_id in it
            full_url = f"https://www.transfermarkt.us{href}"
            team_urls.append(full_url)
    return list(set(team_urls))  # remove duplicates

In [49]:
def extract_address(team_url):
    res = requests.get(team_url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    # Get club name
    title_tag = soup.find("h1", class_="data-header__headline-wrapper data-header__headline-wrapper--oswald")
    club_name = title_tag.get_text(strip=True) if title_tag else "Unknown Club"

    # Set up default output
    out = {"club": club_name, "url": team_url, "street": None, "zip_city_state": None, "country": None}

    # Get street (inside the address block)
    address_block = soup.find("div", itemprop="address")
    if address_block:
        street_tag = address_block.find(itemprop="streetAddress")
        if street_tag:
            out["street"] = street_tag.get_text(strip=True)

    # Get city+state+zip and country (outside the address block)
    locality_tag = soup.find(itemprop="addressLocality")
    country_tag = soup.find(itemprop="addressCountry")

    if locality_tag:
        out["zip_city_state"] = locality_tag.get_text(strip=True)
    if country_tag:
        out["country"] = country_tag.get_text(strip=True)

    return out


In [50]:
league_urls = {
    "MLS": "https://www.transfermarkt.us/major-league-soccer/startseite/wettbewerb/MLS1",
    "USLC": "https://www.transfermarkt.us/usl-championship/startseite/wettbewerb/USL",
    "USL1": "https://www.transfermarkt.us/usl-league-one/startseite/wettbewerb/USC3"
}

all_data = []

for league, league_url in league_urls.items():
    print(f"Scraping {league} team URLs...")
    team_urls = get_team_urls(league_url)
    print(f"Found {len(team_urls)} teams.")

    for url in team_urls:
        try:
            info = extract_address(url)
            info["league"] = league
            all_data.append(info)
            print(f"✅ Scraped {info['club']}")
        except Exception as e:
            print(f"❌ Failed for {url}: {e}")
        time.sleep(1.5)  # be polite

# STEP 4: Save as CSV
df = pd.DataFrame(all_data)

Scraping MLS team URLs...
Found 30 teams.
✅ Scraped Inter Miami CF
✅ Scraped Real Salt Lake City
✅ Scraped Los Angeles Galaxy
✅ Scraped CF Montréal
✅ Scraped D.C. United
✅ Scraped New York Red Bulls
✅ Scraped Austin FC
✅ Scraped New York City FC
✅ Scraped Vancouver Whitecaps FC
✅ Scraped Atlanta United FC
✅ Scraped Charlotte FC
✅ Scraped Portland Timbers
✅ Scraped Unknown Club
✅ Scraped Unknown Club
✅ Scraped Unknown Club
✅ Scraped Unknown Club
✅ Scraped Columbus Crew
✅ Scraped Unknown Club
✅ Scraped Chicago Fire FC
✅ Scraped Unknown Club
✅ Scraped Houston Dynamo FC
✅ Scraped Unknown Club
✅ Scraped Unknown Club
✅ Scraped Unknown Club
✅ Scraped Philadelphia Union
✅ Scraped Unknown Club
✅ Scraped New England Revolution
✅ Scraped Unknown Club
✅ Scraped Unknown Club
✅ Scraped Colorado Rapids
Scraping USLC team URLs...
Found 0 teams.
Scraping USL1 team URLs...
Found 14 teams.
✅ Scraped Unknown Club
✅ Scraped Texoma FC
✅ Scraped Unknown Club
✅ Scraped Unknown Club
✅ Scraped Unknown Club
✅ Sc

In [48]:
all_data

[{'club': 'Inter Miami CF',
  'url': 'https://www.transfermarkt.us/inter-miami-cf/startseite/verein/69261/saison_id/2024',
  'street': '1350 NW 55th St',
  'league': 'MLS'},
 {'club': 'Real Salt Lake City',
  'url': 'https://www.transfermarkt.us/real-salt-lake-city/startseite/verein/6643/saison_id/2024',
  'street': '9526 South State Street',
  'league': 'MLS'},
 {'club': 'Los Angeles Galaxy',
  'url': 'https://www.transfermarkt.us/los-angeles-galaxy/startseite/verein/1061/saison_id/2024',
  'street': '18400 Avalon Blvd',
  'league': 'MLS'},
 {'club': 'CF Montréal',
  'url': 'https://www.transfermarkt.us/club-de-foot-montreal/startseite/verein/4078/saison_id/2024',
  'street': '4750 rue Sherbrooke Est',
  'league': 'MLS'},
 {'club': 'D.C. United',
  'url': 'https://www.transfermarkt.us/d-c-united/startseite/verein/2440/saison_id/2024',
  'street': '2400 East Capitol Street',
  'league': 'MLS'},
 {'club': 'New York Red Bulls',
  'url': 'https://www.transfermarkt.us/new-york-red-bulls/st

In [ ]:
print(df)

                      club                                                url  \
0           Inter Miami CF  https://www.transfermarkt.us/inter-miami-cf/st...   
1      Real Salt Lake City  https://www.transfermarkt.us/real-salt-lake-ci...   
2       Los Angeles Galaxy  https://www.transfermarkt.us/los-angeles-galax...   
3             Unknown Club  https://www.transfermarkt.us/club-de-foot-mont...   
4              D.C. United  https://www.transfermarkt.us/d-c-united/starts...   
5             Unknown Club  https://www.transfermarkt.us/new-york-red-bull...   
6             Unknown Club  https://www.transfermarkt.us/austin-fc/startse...   
7             Unknown Club  https://www.transfermarkt.us/new-york-city-fc/...   
8   Vancouver Whitecaps FC  https://www.transfermarkt.us/vancouver-whiteca...   
9             Unknown Club  https://www.transfermarkt.us/atlanta-united-fc...   
10            Unknown Club  https://www.transfermarkt.us/charlotte-fc/star...   
11            Unknown Club  

In [ ]:
df.to_csv("mls_team_addresses.csv", index=False)